In [5]:
#imports
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [6]:
#training data
df_train=pd.read_csv('/content/drive/My Drive/minor/hindi_dataset/hindi_dataset.tsv',sep='\t', encoding="utf-8")
df_train=df_train[df_train.task_3!='NONE']
#balancing training data by redundancy
# d1=df_train[df_train.task_3==0]
# print(len(d1))
# d2=df_train[df_train.task_3==1]
# print(len(d2))
# d1=pd.concat([d1,d1,d1,d1,d1,d1],ignore_index=True)
# print(len(d1))
# df_train=pd.concat([d1,d2],ignore_index=True)
df_train.head()

,text_id,text,task_1,task_2,task_3
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,HOF,PRFN,UNT
2,hasoc_hi_164,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,HOF,PRFN,TIN
18,hasoc_hi_6865,"#नीच'समानार्थी शब्द #मोदी' दिला तर चालेल की, ल...",HOF,OFFN,TIN
20,hasoc_hi_3763,इस मादरचोद को धुण्डके गांड मे गोली मरो @Uppolice,HOF,PRFN,UNT
22,hasoc_hi_5898,You cry in front of ur god Of being deceived ...,HOF,PRFN,UNT


In [7]:
#training and validation split
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_train['text'], df_train['task_3'],test_size=0.15,random_state=140) 

In [8]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder() #label encoder
train_y = encoder.fit_transform(train_y) #encoding label for training data
valid_y = encoder.fit_transform(valid_y) #encoding label for validation data

In [9]:
#testing data
df_test=pd.read_csv('/content/drive/My Drive/minor/hindi_dataset/hasoc2019_hi_test_gold_2919.tsv', sep='\t',encoding="utf-8") 
df_test=df_test[df_test.task_3!='NONE']
test_y = encoder.fit_transform(df_test['task_3'])#encoding the label of testing data

In [10]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df_train['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)  #get count vector features for training data
xvalid_count =  count_vect.transform(valid_x)  #get count vector features for validation data
x_count =  count_vect.transform(df_test['text'])  #get count vector features for testing data

In [11]:
def model(classifier, feature_vector_train, label, feature_vector_valid,feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions_valid = classifier.predict(feature_vector_valid)

    # predict the labels on testing dataset
    predictions_test = classifier.predict(feature_vector_test)

    print("classification report for validation")
    print(metrics.classification_report(predictions_valid,valid_y))
    print("classification report for testing")
    print(metrics.classification_report(predictions_test,test_y))


In [12]:
# calling the xgboost classifier model for training ,validation and testing
model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc(),x_count.tocsc())

classification report for validation
              precision    recall  f1-score   support

           0       0.92      0.66      0.77       328
           1       0.18      0.56      0.27        43

    accuracy                           0.65       371
   macro avg       0.55      0.61      0.52       371
weighted avg       0.83      0.65      0.71       371

classification report for testing
              precision    recall  f1-score   support

           0       0.95      0.90      0.92       572
           1       0.10      0.18      0.12        33

    accuracy                           0.86       605
   macro avg       0.52      0.54      0.52       605
weighted avg       0.90      0.86      0.88       605

